## DV2599 Assignment 2
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [65]:
# Import packages
import pandas as pd
import statistics as stat
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Load dataset
df = pd.read_csv('spambase/spambase.data', delimiter=',', header=None)

# Add headers to columns to increase clarity and comprehensibility
headers = []
with open('spambase/spambase.names', 'r') as file:
    N = [line for line in file]
    headers = [x[:x.find(':')] for x in N[33:]]

headers.append("spam")  # Spam column is last, 1 = spam, 0 = not spam
df.columns = headers

# Split data into x and y
y = df['spam']
x = df.drop(columns='spam')

df

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


#### 0. Pre-process

In [52]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Initialize standard scaler
scaler = StandardScaler()

# Fit and transform train set
x_train_scaled = scaler.fit_transform(x_train)

# Transform test set
x_test_scaled = scaler.transform(x_test)

# Turn back into pandas dataframes
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)

# Describe
x_train_scaled.describe().round(2)


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
count,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,...,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00
mean,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.35,-0.17,-0.55,-0.05,-0.46,-0.36,-0.29,-0.25,-0.32,-0.36,...,-0.10,-0.16,-0.49,-0.15,-0.38,-0.30,-0.10,-0.13,-0.25,-0.45
25%,-0.35,-0.17,-0.55,-0.05,-0.46,-0.36,-0.29,-0.25,-0.32,-0.36,...,-0.10,-0.16,-0.49,-0.15,-0.38,-0.30,-0.10,-0.11,-0.23,-0.40
50%,-0.35,-0.17,-0.55,-0.05,-0.46,-0.36,-0.29,-0.25,-0.32,-0.36,...,-0.10,-0.16,-0.26,-0.15,-0.38,-0.30,-0.10,-0.09,-0.19,-0.30
75%,-0.35,-0.17,0.29,-0.05,0.10,-0.36,-0.29,-0.25,-0.32,-0.12,...,-0.10,-0.16,0.17,-0.15,0.08,-0.11,-0.10,-0.04,-0.05,-0.03
max,14.89,10.70,9.65,34.92,14.15,20.76,18.13,27.21,18.16,27.36,...,34.16,18.91,33.65,35.77,28.27,23.25,42.09,36.71,47.84,24.67


#### 1. Stratified ten-fold cross-validation

In [68]:
# Initialize ten-fold cross-validation
skf = StratifiedKFold(n_splits=10)

# Initialize models
rf = RandomForestClassifier()
lr = LogisticRegression()
nb = GaussianNB()

models = [("Random Forest", rf), ("Logistic Regression", lr), ("Gaussian NB", nb)]
df_scores = pd.DataFrame({'Fold':[1,2,3,4,5,6,7,8,9,10,'avg', 'stdev']})

for model_name, model in models:
    cv_score = cross_val_score(model, x_train_scaled, y_train, cv=skf, scoring="accuracy")
    cv_score = np.append(cv_score, [stat.mean(cv_score), stat.stdev(cv_score)])
    df_scores[model_name] = cv_score
display(df_scores)


,Fold,Random Forest,Logistic Regression,Gaussian NB
0,1,0.961957,0.967391,0.809783
1,2,0.948370,0.899457,0.809783
2,3,0.967391,0.921196,0.804348
3,4,0.956522,0.937500,0.796196
4,5,0.964674,0.945652,0.842391
5,6,0.929348,0.921196,0.774457
6,7,0.934783,0.913043,0.804348
7,8,0.948370,0.918478,0.798913
8,9,0.945652,0.915761,0.858696
9,10,0.959239,0.934783,0.793478


#### 2. Results

#### 3. Friedman test

#### 4. Significant average differences

##### Nemenyi test